# Quantifying Business Decisions
## Author: Snigdhayan Mahanta

Many business decisions are based on an estimate for the mean value of a business parameter, e.g., average number of monthly labor hours available during a calendar year. Some decision-makers take into consideration additionally the volatility, which can be measured in terms of standard deviation. Nevertheless, mean and standard deviation can only explain a small part of the entire story. Therefore, it can be helpful to analyze further and try to estimate the "expected deviation" between the actual time series dataset and the predicted time series dataset.

`Minkowski distance` is a generalizaton of the `Euclidean distance` between two time series datasets. I created a function to generate a couple of random time series datasets that have the same mean and standard deviation. The first time series dataset can be regarded as the predicted dataset and the second one as the actual dataset. I used `Monte Carlo Simulation` using the `MonteCarlo` package of `R` to compute the average `Minkowski distance` between the two time series datasets. If the `Minkowski distance` can be correlated with the financial impact of a business decision, then future business decisions can take into account this inherent uncertainty of the financial impact of the decision. 

In [1]:
library(MonteCarlo, quietly = TRUE)

In [2]:
# Define a function to create a curated time series dataset
create_ts <- function(size, mean, stdev) { 
    data <- rnorm(size, mean = mean, sd = stdev)
    ts <- ts(data, start = c(2012, 1), frequency = 12)
    return(ts)
}

In [3]:
# Define a function that calculates the Minkowski distance between two random time series datasets
compute_Minkowski <- function(size, mean, stdev, root) {
    ts_forecast <- create_ts(size, mean, stdev)
    ts_actual <- create_ts(size, mean, stdev)
    
    Minkowski <- (sum((ts_actual - ts_forecast)^root))^(1/root) # if root = 2, then Minkowski = Euclidean
    return(list("Minkowski_dist" = Minkowski))
}

In [4]:
# Define parameter grid
size <- 100 # no. of data points
mean <- 0
stdev <- 10
root <- c(2, 10, 100) # if root = 2, then Minkowski = Euclidean

In [5]:
# Collect parameter grids in list
param_list=list("size" = size, "mean" = mean, "stdev" = stdev, "root" = root)

In [6]:
# Run simulation
repetitions <- 100
MC_result<-MonteCarlo(func = compute_Minkowski, nrep = repetitions, param_list = param_list)
summary(MC_result)

Grid of  3  parameter constellations to be evaluated. 
 
Progress: 
 
  |======================================================================| 100%
 
Simulation of function: 

function(size, mean, stdev, root) {
    ts_forecast <- create_ts(size, mean, stdev)
    ts_actual <- create_ts(size, mean, stdev)
    
    Minkowski <- (sum((ts_actual - ts_forecast)^root))^(1/root) # if root = 2, then Minkowski = Euclidean
    return(list("Minkowski_dist" = Minkowski))
}
<bytecode: 0x000000001f2ca628>

Required time: 0.35 secs for nrep = 100  repetitions on 1 CPUs 

Parameter grid: 

  size : 100 
  mean : 0 
 stdev : 10 
  root : 2 10 100 

 
1 output arrays of dimensions: 1 1 1 3 100

In [7]:
# Aggregate results
return1 <- numeric(repetitions) 
return2 <- numeric(repetitions) 
return3 <- numeric(repetitions) 

for (i in c(1:repetitions)) {
    return1[i] <- MC_result$results$Minkowski_dist[3*(i-1)+1]
    return2[i] <- MC_result$results$Minkowski_dist[3*(i-1)+2]
    return3[i] <- MC_result$results$Minkowski_dist[3*(i-1)+3]
}

In [8]:
print(paste0("The mean Minkowski distance (p=2) is ", round(mean(return1), 2)))

[1] "The mean Minkowski distance (p=2) is 140.63"


In [9]:
print(paste0("The mean Minkowski distance (p=10) is ", round(mean(return2), 2)))

[1] "The mean Minkowski distance (p=10) is 41.35"


In [10]:
print(paste0("The mean Minkowski distance (p=100) is ", round(mean(return3), 2)))

[1] "The mean Minkowski distance (p=100) is 38.7"


In [11]:
print(paste0("The standard deviation of Minkowski distance (p=2) measurements is ", round(sd(return1), 2)))

[1] "The standard deviation of Minkowski distance (p=2) measurements is 9.97"


In [12]:
print(paste0("The standard deviation of Minkowski distance (p=10) measurements is ", round(sd(return2), 2)))

[1] "The standard deviation of Minkowski distance (p=10) measurements is 5.39"


In [13]:
print(paste0("The standard deviation of Minkowski distance (p=100) measurements is ", round(sd(return3), 2)))

[1] "The standard deviation of Minkowski distance (p=100) measurements is 5.37"
